In [ ]:
object_name = "yasone2"

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import astroalign

In [ ]:
from astropy.nddata import CCDData
from astropy.stats import mad_std

from ccdproc import ImageFileCollection
import ccdproc


In [ ]:
import sys
sys.path.append("../imaging/")

In [ ]:
from phot_utils import swap_byteorder

In [ ]:
from convenience_functions import show_image, show_images, show_image_residual

In [ ]:
import astroalign

In [ ]:
from pathlib import Path

In [ ]:
import arya
plt.rcParams["figure.dpi"] = 300

In [ ]:
import os

## Setup

In [ ]:
filters = ["g", "r", "i"]

In [ ]:
DATADIR = Path(f"./{object_name}")

In [ ]:
# for filt in ["g", "r", "i"]:
#     for cls in ["obj", "std"]:
#         filename = DATADIR / f"reduced/stacked_{cls}_{filt}.fits"
#         if filename.is_file():
#             os.remove(filename)

In [ ]:
def get_image_files(filt):
    imgs = []
    for f in DATADIR.glob(f"img_{filt}_*"):
        imgs.append(swap_byteorder(CCDData.read(f / "nobkg.fits").data))
    return imgs

In [ ]:
imgs_obj = {filt: get_image_files(filt) for filt in filters}

In [ ]:
def align_all(images):
    image_0 = images[0]
    mask_combined = np.full_like(image_0, False, dtype=bool)

    images_aligned = [image_0]
    for i in range(1, len(images)):
        print(f"aligning image {i} / {len(images)-1}")
        image_aligned, mask = astroalign.register(images[i], image_0, max_control_points=60)
        mask_combined = mask_combined | mask
        images_aligned.append(image_aligned)

    return images_aligned, mask_combined

In [ ]:
imgs_mask_obj_aligned = {filt: align_all(imgs) for filt, imgs in imgs_obj.items()}


In [ ]:

imgs_obj_aligned = {filt: imgs for filt, (imgs, mask) in imgs_mask_obj_aligned.items()}
mask_obj_aligned = {filt: mask for filt, (imgs, mask) in imgs_mask_obj_aligned.items()}


The above plot shows an example of the alignment process

In [ ]:
def combine_images(filenames, **kwargs):
    return ccdproc.combine(filenames, 
                           method="median",
                           sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5, 
                           sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std, 
                           mem_limit=1e9,
                           **kwargs
                            )

In [ ]:
imgs_obj_stacked = {filt: combine_images([CCDData(img, unit="adu") for img in imgs]) for filt, imgs in imgs_obj_aligned.items()}

In [ ]:
for filt in filters:
    imgs_obj_stacked[filt].mask = mask_obj_aligned[filt]


In [ ]:
# for filt in ["g", "r", "i"]:
    # imgs_obj_stacked[f"Sloan_{filt}"].write(DATADIR / f"reduced/stacked_obj_{filt}.fits")


# Plots

The above image is an example of an alignment shift on an image

In [ ]:
for filt in filters:
    show_image(imgs_obj_stacked[filt], clabel="ADU", log=True)
    plt.title(filt)

## Save residual images

In [ ]:
DATADIR

In [ ]:
(DATADIR / "residual").mkdir(exist_ok=True)
(DATADIR / "residual/object").mkdir(exist_ok=True)
(DATADIR / "residual/stds").mkdir(exist_ok=True)

In [ ]:
# for filt in filters:
#     for img, name in zip(imgs_obj_aligned[filt], files_obj[filt]):
#         img_processed = ccdproc.subtract_bias(CCDData(img, unit="adu"), imgs_obj_stacked[filt])
#         filename = DATADIR / "residual/object" / name
#         img_processed.write(filename)

In [ ]:
# for filt in filters:
#     for img, name in zip(imgs_std_aligned[filt], files_std[filt]):
#         img_processed = ccdproc.subtract_bias(CCDData(img, unit="adu"), imgs_std_stacked[filt])
#         filename = DATADIR / "residual/stds" / name
#         img_processed.write(filename)

In [ ]:

show_images(imgs_obj_aligned[filters[1]])

In [ ]:
imgs_obj_stacked["g"].shape

In [ ]:
imgs_obj_stacked["g"].data - imgs_obj_aligned["g"][0]

In [ ]:
imgs_obj_aligned["g"][0].shape

In [ ]:
for filt in filters:
    for i in range(len(imgs_obj_aligned[filt])):
        show_image_residual(imgs_obj_aligned[filt][i], imgs_obj_stacked[filt].data)